# Chargement des données

In [20]:
# code inspiré de https://git.regardscitoyens.org/regardscitoyens/sprint-scrutins/blob/master/votes_an/ancsv.py

import glob, json, csv, collections

# recupere les députés

députés = list(csv.DictReader(open('liste_deputes_libre_office.csv')))
députés_par_id = {int(député["identifiant"]): député for député in députés}


# recuperer les votes

votes_par_députés = {id_député: collections.defaultdict(list) for id_député in députés_par_id}

for fichier in glob.glob('scrutins/*.json'):
    
    aucun_vote = True
    
    scrutin = json.load(open(fichier))['scrutin']
    
    if scrutin['numero'] == 1:
        print(scrutin)

    id_scrutin = scrutin['numero']

    groupes = scrutin['ventilationVotes']['organe']['groupes']['groupe']

    for groupe in groupes:
        dn = groupe['vote']['decompteNominatif']
        
        for position in ('pour', 'contre', 'abstention'):
            votants_position = dn['%ss' % position]
            
            if not votants_position:
                continue
            
            votants = votants_position['votant']

            if not isinstance(votants, list):
                votants = [votants]

            for votant in votants:
                votant_id = int(votant['acteurRef'][2:])
                
                """
                if votant.get('parDelegation', 'false') == 'true':
                    continue
                """

                aucun_vote = False
                if votant_id in votes_par_députés:
                    votes_par_députés[votant_id][scrutin['dateScrutin']].append(position)
    
    if aucun_vote:
        print(scrutin['dateScrutin'], '\t', scrutin['numero'])


In [2]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [27]:
import datetime, statistics

# grouper par semaine

votes_par_mois = collections.defaultdict(dict)

for député, dates in votes_par_députés.items():
    for date, votes in dates.items():
        année, mois, jour = date.split('-')
        mois_complet = f"{année}-{mois}"
        if député not in votes_par_mois[mois_complet]:
            votes_par_mois[mois_complet][député] = [votes]
        votes_par_mois[mois_complet][député] += votes

def moyenne_votes(votes):
    return statistics.mean([len(votes.get(député, [])) for député in députés_par_id])

from bokeh.io import show
from bokeh.plotting import figure
        
tout_les_mois = sorted(votes_par_mois.keys())

def compte_votes(député):
    return sum([len(mois.get(député, [])) for mois in votes_par_mois.values()])

députés_les_moins_présents = sorted(votes_par_députés.keys(),
                                    key=compte_votes)

def affiche_député(député):
    votes = [len(votes_par_mois[mois].get(député, [])) for mois in tout_les_mois]
    votes_moyenne = [moyenne_votes(votes_par_mois[mois]) for mois in tout_les_mois]

    p = figure(x_range=tout_les_mois, plot_width=700, plot_height=400,
               title=f"Votes par mois pour {députés_par_id[député]['Prénom']} {députés_par_id[député]['Nom']} avec {compte_votes(député)} votes",
               toolbar_location=None, tools="")

    p.step(tout_les_mois, votes_moyenne, line_width=2, line_color="red", mode="before", alpha=0.8, legend_label='Moyenne de tous les députés')
    p.vbar(x=tout_les_mois, top=votes, width=1, alpha=0.5)

    p.y_range.start = 0
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.xaxis.major_label_orientation = "vertical"
    show(p)


# Les 10 moins présents

In [26]:
for député in députés_les_moins_présents[:10]:
    affiche_député(député)

# Les 10 plus présents

In [14]:
for député in députés_les_moins_présents[-10:]:
    affiche_député(député)

In [6]:
# todo: trous dans les mois
# plot line not aligned, ligne date debut mandat